* Filter:
    - Importance: High => Low
    - Medium questions only

## Debounce

* main pitfall is being able to access `this` from inside the callback
* there are 2 ways of doing this:
    1. have a variable initialized with the `this` value inside the returned debounced function
        * then you use `.call(thisArgs, ...args)`
    2. use an arrow function inside the setTimeout
        * the `this` inside the setTimeout will be the same one in the outer function
        * so you skip the step of having to initialize and use the outer function's `this` value
* we should NOT return an arrow function inside the debounce()
    - reason being, it's bound to the place it was created and not where it's called
* for example:

```
const increment = debounce(function(num) {
    console.log(this);
    this.val += num;
}, 10);

const obj = {
    val: 2,
    increment,
};

obj.increment(5);
```
* if you tried this in jsfiddle in which the debounce returned an arrow function instead of a function(), `this` would be the window object and not obj

In [ ]:
// my solution

/**
 * @param {Function} func
 * @param {number} wait
 * @return {Function}
 * 
 * must wait until after x milliseconds after
 * last debounced function was last called
 * in order to actually execute the real function
 */
export default function debounce(func, wait) {
  // need to settimeout before calling the function
  let timer;

  // every time we call debounced function
  // it returns a function

  return function(...args) {
    // if it's called, we clear the timer
    clearTimeout(timer);
    const that = this;
    // set a new timer
    timer = setTimeout(function() {
      func.call(that, ...args);
    }, wait);
  }
}

In [ ]:
// actual solution

/**
 * @callback func
 * @param {number} wait
 * @return {Function}
 */
export default function debounce(func, wait = 0) {
  let timeoutID = null;
  return function (...args) {
    clearTimeout(timeoutID);

    timeoutID = setTimeout(() => {
      timeoutID = null; // Not strictly necessary but good to include.
      // Has the same `this` as the outer function's
      // as it's within an arrow function.
      func.apply(this, args);
    }, wait);
  };
}

## Debounce II

* __creating static methods is exactly the same as creating methods on an object__
    - for this question, we could just use dot notation and assign the methods onto the function itself
    - remember that pretty much everything in JS is an object
* flush does not have access to the `this` value or parameters from the returned function
    - instead, you must create a function-level variables and assign them the `this` and the parameters from inside the returned function
* if a debounced function has already been invoked, calling flush() should not do anything!
    - to implement this, we can check if timer is null and if it is, we can return immediately
    - we should also set timer to null whenever we cancel or finish invoking the timer from the setTimeout

In [ ]:
// my solution

/**
 * @param {Function} func
 * @param {number} wait
 * @return {Function}
 * 
 * has to return an object with 2 methods:
 * 1) cancel => cancels pending invocations (probably just cleartimeout)
 * 2) flush => immediately invokes delayed invocations
 *  - also clears timeout but calls the function
 * 
 * pitfall: how to call `this` inside function?
 * probably just use a variable again
 */
export default function debounce(func, wait) {
  let timer = null;
  let context = null;
  let params = [];

  const myDebounce = function(...args) {
    context = this;
    params = args;
    clearTimeout(timer);
    timer = setTimeout(() => {
      timer = null;
      func.call(this, ...args);
    }, wait);
  };

  myDebounce.cancel = () => {
    clearTimeout(timer);
    timer = null;
  }

  myDebounce.flush = () => {
    if (timer === null || context === null) return;
    myDebounce.cancel();
    func.call(context, ...params);
  }

  return myDebounce;
}

In [ ]:
// actual solution

/**
 * @param {Function} func
 * @param {number} wait
 * @return {Function}
 */
export default function debounce(func, wait = 0) {
  let timeoutId = null;
  let context = undefined;
  let argsToInvoke = undefined;

  function clearTimer() {
    clearTimeout(timeoutId);
    timeoutId = null;
  }

  function invoke() {
    // Don't invoke if there's no pending callback.
    if (timeoutId == null) {
      return;
    }

    clearTimer();
    func.apply(context, argsToInvoke);
  }

  function fn(...args) {
    clearTimer();
    argsToInvoke = args;
    context = this;
    timeoutId = setTimeout(function () {
      invoke();
    }, wait);
  }

  fn.cancel = clearTimer;
  fn.flush = invoke;
  return fn;
}


## Promise.all()

* just remember to have a counter for the number of items that we've processed so far
    - if it matches # of items in the iterable, then that means nothing rejected and we can return the entire result array

In [ ]:
/**
 * @param {Array} iterable
 * @return {Promise<Array>}
 */
export default function promiseAll(iterable) {
  return new Promise((resolve, reject) => {
    if (iterable.length === 0) resolve([]);
    const result = new Array(iterable.length);
    let counter = 0;

    iterable.forEach(async (item, index) => {
      try {
        const value = await item;
        result[index] = value;
        counter++;
        if (counter === iterable.length) {
          resolve(result);
        }
      } catch(e) {
        reject(e);
      }
    });

  });
}

## Throttle

* remember to check the boolean's state
    - if you canCall = false, then we know we're currently throttling the function calls
* use an anonymous function `function(){}` instead of an arrow function because we can use the `this` value if the function is used as a method
    - then you can just pass in the `this` value into `func.apply()` or `func.call()` to bind `this` to it
    - if you needed `this` in the setTimeout, you would just create a variable and assign `this` to it or use an arrow function as the callback for the setTimeout since it takes the `this` value of the place where it was created

In [ ]:
// my solution

/**
 * @callback func
 * @param {number} wait
 * @return {Function}
 * 
 * invoked IMMEDIATELY
 * has to wait X milliseconds before being able to be called again
 * 
 * should return a function with a throttled invocation of the callback
 */
export default function throttle(func, wait) {
  // keeps track of whether we can call or not
  let canCall = true;
  return function(...args) {
    // have to make sure that the function can call `this`?
    // make the call
    if (!canCall) return;

    func.apply(this, args);
    canCall = false;

    setTimeout(() => {
      canCall = true;
    }, wait);
  }
}

In [ ]:
// actual solution

/**
 * @callback func
 * @param {number} wait
 * @return {Function}
 */
export default function throttle(func, wait = 0) {
  let shouldThrottle = false;

  return function (...args) {
    if (shouldThrottle) {
      return;
    }

    shouldThrottle = true;
    setTimeout(function () {
      shouldThrottle = false;
    }, wait);

    func.apply(this, args);
  };
}

## Promise.allSettled()

* very similar to Promise.all()
* just have to make sure we don't accidentally increment count instead of counter

In [ ]:
/**
 * @param {Array} iterable
 * @return {Promise<Array<{status: 'fulfilled', value: *}|{status: 'rejected', reason: *}>>}
 */
export default function promiseAllSettled(iterable) {
  return new Promise(resolve => {
    if (iterable.length === 0) resolve([]);

    const result = new Array(iterable.length);
    let counter = 0;

    iterable.forEach(async (item, index) => {
      try {
        const value = await item;
        result[index] = {
          status: 'fulfilled',
          value,
        };
      } catch(e) {
        result[index] = {
          status: 'rejected',
          reason: e,
        }
      }
      counter++;
      if (counter === iterable.length) {
        resolve(result);
      }
    });
  })
}

## Digital Clock

* using `setInterval` was the correct solution but the interval itself could've been better
    - if we used 1000ms/1second, then the clock might be off if we pulled the date right on the last millisecond of a second
    - if we used a very small interval, then it would be expensive to keep pulling the date and updating the state
    - therefore, around 100ms is a good interval because we wouldn't be off by too much with 1000ms and it is not that expensive to pull at that interval
* instead of using a `div`, we should instead use a `time` element with the `datetime` attribute set to the current date in 24-hour format for accessibility reasons
    - helps screen readers out

In [ ]:
// app.js
import Clock from './Clock';

export default function App() {
  return <Clock />;
}

// clock.js
/**
 * how do i create a render of this? 
 * 
 *  _
 * | |
 *  -
 * | |
 *  _
 * 
 * just need a box
 * color in the border depending on which number it is
 * 
 * time.map(digit => {
 *  < segment number={digit} />
 * })
 * 
 * depending on what number is equal to,
 * we assign a class to it
 */

import { useState, useEffect, useRef } from 'react';

function Segment({ digit }) {
  return (
    <div className="segmentWrapper">
      <div className={`segment top-${digit}`} />
      <div className={`segment bottom-${digit}`} />
    </div>
  );
}

export default function Clock() {
  const [time, setTime] = useState(() => new Date());
  const timeString = time.toTimeString().split(" ")[0];

  useEffect(() => {
    const timer = setInterval(() => {
      setTime(() => new Date());
    }, 1000);

    return () => {
      clearInterval(timer);
    }
  }, []);

  const segments = timeString.split("").map((digit, index) => {
    return isNaN(digit) ? (
      <span className="colon" key={`colon-${index}`}>
        <p className="colonText">:</p>
      </span>
    ): (
      <Segment digit={digit} key={index} />
    );
  })
  return (
    <div className="clock">
      {segments}
    </div>
  );
}


// styles.css
body {
  font-family: sans-serif;
}

.clock {
  height: 150px;
  width: 100%;
  display: flex;
  flex-direction: row;
  justify-content: center;
  align-items: center;
  background-color: black;
  border: 7px solid gray;
  border-radius: 0.5rem;
}

.segmentWrapper {
  height: 75%;
  width: 50px;
  display: flex;
  flex-direction: column;
}

div.segmentWrapper:nth-of-type(even) {
  margin-left: 0.75rem;
}

.segment {
  flex: 1;
  border: 2px solid white;
}


.colon {
  height: 75%;
  display: flex;
  justify-content: center;
  align-items: center;
  margin-left: 0.5rem;
  margin-right: 0.5rem;
}

.colonText {
  color: white;
  font-size: 3rem;
}

.top-0 { border-bottom: none; }
.bottom-0 {border-top: none; }
.top-1, .bottom-1 {
  border-top: none;
  border-bottom: none;
  border-left: none;
}

.top-2 {
  border-left: 0;
}
.bottom-2 {
  border-top: 0;
  border-right: 0;
}

.top-3 {
  border-left: none;
}
.bottom-3 {
  border-top: none;
  border-left: none;
}

.top-4 {
  border-top: none;
}
.bottom-4 {
  border-top: none;
  border-left: none;
  border-bottom: none;
}

.top-5 {
  border-right: none;
}
.bottom-5 {
  border-top: none;
  border-left: none;
}

.top-6 {
  border-top: none;
  border-right: none;
}
.bottom-6 {
  border-top: none;
}

.top-7 {
  border-left: none;
  border-bottom: none;
}
.bottom-7 {
  border-top: none;
  border-left: none;
  border-bottom: none;
}

.bottom-8 {
  border-top: none;
}

.bottom-9 {
  border-top: none;
  border-left: none;
}

In [ ]:
// actual solution

// app.js
import Clock from './Clock';

export default function App() {
  return (
    <div className="wrapper">
      <Clock />
    </div>
  );
}

// clock.js
import { useEffect, useState } from 'react';

const ALL_SIDES = [
  'digit-square-border-top',
  'digit-square-border-left',
  'digit-square-border-right',
  'digit-square-border-bottom',
];

const NUMBER_TO_CLASSES = {
  0: {
    top: [
      'digit-square-border-top',
      'digit-square-border-left',
      'digit-square-border-right',
    ],
    bottom: [
      'digit-square-border-bottom',
      'digit-square-border-left',
      'digit-square-border-right',
    ],
  },
  1: {
    top: ['digit-square-border-right'],
    bottom: ['digit-square-border-right'],
  },
  2: {
    top: [
      'digit-square-border-top',
      'digit-square-border-right',
      'digit-square-border-bottom',
    ],
    bottom: [
      'digit-square-border-top',
      'digit-square-border-left',
      'digit-square-border-bottom',
    ],
  },
  3: {
    top: [
      'digit-square-border-top',
      'digit-square-border-right',
      'digit-square-border-bottom',
    ],
    bottom: [
      'digit-square-border-top',
      'digit-square-border-right',
      'digit-square-border-bottom',
    ],
  },
  4: {
    top: [
      'digit-square-border-left',
      'digit-square-border-right',
      'digit-square-border-bottom',
    ],
    bottom: [
      'digit-square-border-right',
      'digit-square-border-top',
    ],
  },
  5: {
    top: [
      'digit-square-border-top',
      'digit-square-border-left',
      'digit-square-border-bottom',
    ],
    bottom: [
      'digit-square-border-top',
      'digit-square-border-right',
      'digit-square-border-bottom',
    ],
  },
  6: {
    top: [
      'digit-square-border-top',
      'digit-square-border-left',
      'digit-square-border-bottom',
    ],
    bottom: ALL_SIDES,
  },
  7: {
    top: [
      'digit-square-border-top',
      'digit-square-border-right',
    ],
    bottom: ['digit-square-border-right'],
  },
  8: {
    top: ALL_SIDES,
    bottom: ALL_SIDES,
  },
  9: {
    top: ALL_SIDES,
    bottom: [
      'digit-square-border-top',
      'digit-square-border-right',
      'digit-square-border-bottom',
    ],
  },
};

function Digit({ number }) {
  const { top, bottom } = NUMBER_TO_CLASSES[number];
  return (
    <div>
      <div
        className={[
          'digit-square',
          'digit-square-top',
          ...top,
        ].join(' ')}
      />
      <div
        className={[
          'digit-square',
          'digit-square-bottom',
          ...bottom,
        ].join(' ')}
      />
    </div>
  );
}

function Separator() {
  return (
    <div className="separator">
      <div className="separator-dot" />
      <div className="separator-dot" />
    </div>
  );
}

function useCurrentDate() {
  const [date, setDate] = useState(new Date());

  // Kick off the timer.
  useEffect(() => {
    const timer = window.setInterval(() => {
      setDate(new Date());
    }, 100);

    // Clear the timer upon unmount.
    return () => {
      window.clearInterval(timer);
    };
  }, []);

  return date;
}

function padTwoDigit(number) {
  return number >= 10 ? String(number) : `0${number}`;
}

export default function App() {
  const date = useCurrentDate();

  let hours = date.getHours() % 12;
  hours = hours === 0 ? 12 : hours;
  const minutes = date.getMinutes();
  const seconds = date.getSeconds();

  const dateTimeDisplay = `${padTwoDigit(
    date.getHours(),
  )}:${padTwoDigit(minutes)}:${padTwoDigit(seconds)}`;

  // Use a <time> element with `datetime` attribute set
  // to the current time in 24-hour format so that
  // screen readers can read this component.
  return (
    <time className="clock" dateTime={dateTimeDisplay}>
      <Digit number={parseInt(hours / 10, 10)} />
      <Digit number={hours % 10} />
      <Separator />
      <Digit number={parseInt(minutes / 10, 10)} />
      <Digit number={minutes % 10} />
      <Separator />
      <Digit number={parseInt(seconds / 10, 10)} />
      <Digit number={seconds % 10} />
    </time>
  );
}

// styles.css
body {
  font-family: sans-serif;
}

.wrapper {
  display: flex;
  align-items: center;
  justify-content: center;
}

.clock {
  --segment-width: 10px;
  --segment-size: 40px;
  --segment-color: #fff;

  background-color: #000;
  border: 10px solid #ccc;
  border-radius: 10px;
  display: flex;
  gap: 10px;
  padding: 20px;
}

.digit-square {
  border-style: solid;
  border-color: transparent;
  border-width: var(--segment-width);
  box-sizing: border-box;
  height: var(--segment-size);
  width: var(--segment-size);
}

.digit-square-top {
  border-bottom-width: calc(var(--segment-width) / 2);
}

.digit-square-bottom {
  border-top-width: calc(var(--segment-width) / 2);
}

.digit-square-border-top {
  border-top-color: var(--segment-color);
}

.digit-square-border-left {
  border-left-color: var(--segment-color);
}

.digit-square-border-right {
  border-right-color: var(--segment-color);
}

.digit-square-border-bottom {
  border-bottom-color: var(--segment-color);
}

.separator {
  align-items: center;
  display: flex;
  flex-direction: column;
  justify-content: space-evenly;
}

.separator-dot {
  background-color: var(--segment-color);
  border-radius: var(--segment-width);
  height: var(--segment-width);
  width: var(--segment-width);
}